# PART 0: INSTALL (for Colab and New Devices)

In [1]:
!pip install cma

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 230 kB 3.2 MB/s eta 0:00:01


# PART 1: EVALUATION

In [2]:
import cma
from cma.fitness_transformations import EvalParallel2

## TOY EXAMPLE: ROSENBROCK FUNCTION

In [3]:
es = cma.CMAEvolutionStrategy(8 * [0], 0.5)

f = cma.ff.rosen
es.optimize(f)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=439291, Wed Nov 25 12:42:29 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 4.444113787120208e+01 1.0e+00 4.46e-01  4e-01  5e-01 0:00.2
    2     20 3.775374418792205e+01 1.2e+00 3.80e-01  4e-01  4e-01 0:00.2
    3     30 7.845480415050527e+01 1.2e+00 3.38e-01  3e-01  3e-01 0:00.2
  100   1000 5.404711831871216e+00 5.4e+00 4.51e-02  1e-02  4e-02 0:00.5
  200   2000 1.321304421049287e+00 1.6e+01 1.28e-01  2e-02  5e-02 0:00.7
  300   3000 1.351070150502091e-02 4.1e+01 6.55e-02  2e-03  4e-02 0:01.0
  400   4000 1.570917490708622e-09 7.7e+01 8.44e-05  9e-07  2e-05 0:01.3
  459   4590 2.918124787529673e-14 9.2e+01 1.22e-06  5e-09  2e-07 0:01.5


In [10]:
with EvalParallel2(f, number_of_processes=12) as eval:
    while not es.stop():
        X = es.ask()
        es.tell(X, eval(X))
print(es.result[1], es.result[2])

5.488439864559804e-15 4000


## SLIGHTLY BIGGER TOY EXAMPLE: MNIST
Builds an 1-layer neural network of 

In [4]:
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# "GLOBALS"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TRAIN_LOADER = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=2048, shuffle=True, num_workers = 8)

TEST_LOADER = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=2048, shuffle=True, num_workers = 8)

In [ ]:
BOUNDS = [np.array([-8,  2,  2,  2,  2,  16]),
          np.array([-5,  20, 20, 8, 8, 512])]

def mnist_run(hyperparameters, verbose=False):
    """
        Generate a training run for an MNIST classifier with given hyperperameter values
        :param hyperparameters (3, int): 1D array of hyperparameter values
        :return: evaluation metric
    """
    hyperparameters = hyperparameters.astype(int)
    LR = hyperparameters[0] #LEARNING RATE (EXPONENT)
    EP = 3#hyperparameters[1] #EPOCHS
    C1 = hyperparameters[1] #CHANNEL SIZE 1
    C2 = hyperparameters[2] #CHANNEL SIZE 2
    K1 = hyperparameters[3] #KERNEL SIZE 1
    K2 = hyperparameters[4] #KERNEL SIZE 2
    LW = hyperparameters[5] #LINEAR LAYER WIDTH

    class MNIST_Net(nn.Module):
        def __init__(self, C1, K1, C2, K2, LW):
            super(MNIST_Net, self).__init__()
            self.conv1 = nn.Conv2d(1, C1, kernel_size=K1)
            self.w1 = 28-K1+1
            self.conv2 = nn.Conv2d(C1, C2, kernel_size=K2)
            self.w2 = (self.w1-K2-1)//2+1
            self.conv2_drop = nn.Dropout2d(0.25)
            
            self.width2 = self.w2*self.w2*C2
            self.fc1 = nn.Linear(self.width2, LW)
            self.fc2 = nn.Linear(LW, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(F.relu(self.conv2(x)), 2)
            x = self.conv2_drop(x)
            x = x.view(-1, self.width2)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            return F.log_softmax(x)

    def train(network, optimizer, train_loader, epochs, verbose=False):
        network.train()
        for i in range(epochs):
            for batch_idx, (data, target) in enumerate(train_loader):
                data = data.to(DEVICE)
                target = target.to(DEVICE)
                optimizer.zero_grad()
                output = network(data)
                loss = F.nll_loss(output, target)
                loss.backward()
                optimizer.step()

                if batch_idx % 400 == 0 and verbose:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        i, batch_idx * len(data), len(TRAIN_LOADER.dataset),
                        100. * batch_idx / len(train_loader), loss.item()))

                del data, target, output, loss
            
    def test(network, test_loader, verbose=False):
        network.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data = data.to(DEVICE)
                target = target.to(DEVICE)
                output = network(data)
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()

                del data, target, output
        return correct
    
    network = MNIST_Net(C1, K1, C2, K2, LW)
    network.to(DEVICE)
    optimizer = optim.Adadelta(network.parameters(), lr=1.0**LR)

    train(network, optimizer, TRAIN_LOADER, EP, verbose)
    correct = test(network, TEST_LOADER, verbose)
    return len(TEST_LOADER.dataset) - correct.item()

In [ ]:
options = cma.CMAOptions()
options.set('bounds', BOUNDS)

es = cma.CMAEvolutionStrategy([-6, 10, 20, 5, 5, 50], 2)
f = cma.s.ft.IntegerMixedFunction(mnist_run, [1, 2, 3, 4, 5])

#es.optimize(f)
with EvalParallel2(f, number_of_processes=1) as eval: #SET number_of_processes
    while not es.stop():
        X = es.ask()
        es.tell(X, eval(X)) #CHANGE eval() to TRAINING WRAPPER
print(es.result[1], es.result[2])

# PART 2: EVAL FUNCTION WRAPPER

In [ ]:
def launch_run(hyperparameters):
    """
        TODO: write training run code for ASR
            - Unroll Hyperparameter array
            - Generate necessary scripts
            - Run bash scripts
            - Read and evaluate WER/CER
            - Return to CMA-ES
        TODO: integrate with ESPnet and feature extraction model training procedures

        Generate a training run of ASR system with selected hyperparameters
        :param hyperparameters: 1D array of hyperparameter values
        :return: evaluation metric
    """  
    raise NotImplementedError